In [1]:
using LinearAlgebra
# examples from https://www.math.uci.edu/~ttrogdon/105A/html/Lecture23.html

In [2]:
function cgs(A :: Matrix{Float64})
  n, _ = size(A)
  Q = zeros(n, n)

  for i = 1:n
    vi = @view A[:,i]
    ui = copy(vi)
        
    for j = 1:i-1
      uj = @view Q[:,j]
      ui .-= (uj ⋅ vi) * uj
    end

    normalize!(ui)
    Q[:,i] .= ui
  end
  R = Q' * A
  return Q, R
end

cgs (generic function with 1 method)

In [3]:
function mgs(A :: Matrix{Float64})
  n, _ = size(A)
  Q = zeros(n, n)

  for i = 1:n
    vi = @view A[:,i]
    ui = copy(vi)
        
    for j = 1:i-1
      uj = @view Q[:,j]
      ui .-= (uj ⋅ ui) * uj
    end

    normalize!(ui)
    Q[:,i] .= ui
  end
  R = Q' * A
  return Q, R
end

mgs (generic function with 1 method)

In [4]:
n = 200
A = [(i == j ? 1.0001 : 1)/(i+j-1) for i=1:n, j=1:n];

Q, _ = cgs(A); @show norm(I - Q' * Q);
Q, _ = mgs(A); @show norm(I - Q' * Q);

A = rand(n, n)

Q, _ = cgs(A); @show norm(I - Q' * Q);
Q, _ = mgs(A); @show norm(I - Q' * Q);

norm(I - Q' * Q) = 179.8030660935869
norm(I - Q' * Q) = 4.662298863101829e-10
norm(I - Q' * Q) = 6.165972518353212e-12
norm(I - Q' * Q) = 8.51811696192949e-14


In [5]:
A = rand(5, 10)
x = ones(10)
b = A * x
Q, R = cgs(A)

@show norm(A \ b - x);

norm(A \ b - x) = 0.826554255780483


In [6]:
@show norm(R \ (Q' * b) - x);

norm(R \ (Q' * b) - x) = 0.8265542557804835


In [8]:
function householder_hessenberg(A :: Matrix{Float64})
  H = copy(A)
  N = size(A, 1)
  for i=1:N-2
    α = H[i+1:N, i] # FIXME
    σ = norm(α)
    β = 1.0  / (σ * (σ + abs(α[1])))
    expjϕ = α[1] < 0.0 ? -1.0 : 1.0 
    u = copy(α)
    u[1] = expjϕ * (σ + abs(α[1]))
    T = diagm(ones(N))
    T[i+1:N, i+1:N] .= diagm(ones(length(α))) - β * u * u'
    H .= T * H * T
  end
  return H
end

householder_hessenberg (generic function with 1 method)

In [9]:
using LinearAlgebra
function givensrot(A, i, j)
  q = sqrt(A[j,j]^2 + A[i,j]^2);
  c = A[j,j] / q;
  s = A[i,j] / q;
  N = size(A, 1);
  G = diagm(ones(N));
  G[i,i] = c; G[j,j] = c;
  G[j,i] = s; G[i,j] =-s;
  return G
end

givensrot (generic function with 1 method)

In [12]:
function francis(A :: Matrix{Float64}, maxiter)
  H = householder_hessenberg(A)
  N = size(H, 1)
  G = [[0. 0.; 0. 0.] for j=1:N-1]
  λs = zeros(N)
  for k = 1:maxiter
    for j = 1:N-1
      q = sqrt(H[j,j]^2 + H[j+1,j]^2);
      c = H[j,  j] / q;
      s = H[j+1,j] / q;
      G[j] = [c -s; +s c]
      H[j:j+1, j:N] = G[j]' * H[j:j+1, j:N] # => 2 x (N-j)
    end
    for j = 1:N-1
      H[1:j+1, j:j+1] = H[1:j+1, j:j+1] * G[j] # (j) x 2
    end
    λs .= diag(H)
  end
  display(dropalmostzeros(H, 1e-15))  
  return λs
end

francis (generic function with 1 method)

In [13]:
using SparseArrays
dropalmostzeros(A, ϵ) = sparse([abs(aij) > ϵ ? aij : 0.0 for aij in A])

dropalmostzeros (generic function with 1 method)

In [14]:
A = rand(5,5)
λ = francis(A, 1000)

5×5 SparseMatrixCSC{Float64, Int64} with 16 stored entries:
 2.11111  -0.296804  -0.270293   -0.395172  -0.247274
  ⋅       -0.743927  -0.237609    0.111608  -0.130035
  ⋅         ⋅        -0.0576799  -0.214469  -0.123551
  ⋅         ⋅         0.271744   -0.201957   0.0882204
  ⋅         ⋅          ⋅           ⋅         0.0493063

5-element Vector{Float64}:
  2.1111065790557526
 -0.7439269829618946
 -0.057679880666073456
 -0.20195663078463086
  0.0493062881877976

In [15]:
eigvals(A)

5-element Vector{ComplexF64}:
  -0.7439269829618954 + 0.0im
 -0.12981825572535174 - 0.2303839708102201im
 -0.12981825572535174 + 0.2303839708102201im
  0.04930628818779772 + 0.0im
    2.111106579055753 + 0.0im

In [16]:
eigvals([-0.0576799  -0.214469; 0.271744   -0.201957])

2-element Vector{ComplexF64}:
 -0.12981844999999997 - 0.23038379617476895im
 -0.12981844999999997 + 0.23038379617476895im